# Function Calling with Gemini

In [3]:
from google import genai
client = genai.Client()

In [4]:
import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [5]:
get_current_weather("Bangalore")

{'City Name': 'Bangalore',
 'weather': 'broken clouds',
 'temperature': 297.32,
 'unit': 'kelvin'}

### Tool with LLM

- we need to provide the metadata about the tools/tool to the llm
- metadata needs to have 3 components
    - name of the function/tool - should be descriptive, clear
    - description of the function - clear with a sample example
    - arguments: args and its description with example
- OPENAPI format to define metadata

In [6]:
weather_tool = [{"name":"get_current_weather",
                 "description":"this function is used to get/fetch the current weather information for any given city.",
                 "parameters":{"type":"object",
                               "properties":{"city":{"type":'string','description':'name of any location/city. e.g. mumbai, new york'}},
                               "required":['city'],},
                               }]

In [7]:
from google.genai import types
import json
tools = types.Tool(function_declarations=weather_tool)
tool_map = {"get_current_weather":get_current_weather}

config = types.GenerateContentConfig(tools=[tools],
                                     automatic_function_calling=types.FunctionCallingConfig(mode="AUTO"))

In [13]:
def generate_content(prompt:str):
    content = [types.Content(role='user',parts=[types.Part(text=prompt)])]

    first_response = client.models.generate_content(model='gemini-2.0-flash',
                                                    contents=content,
                                                    config=config)
    if first_response.candidates[0].content.parts[0].function_call:
        print("LLM decided to make function call \n",first_response.candidates[0].content.parts[0].function_call)
        content.append(first_response.candidates[0].content)

        tool_call = first_response.candidates[0].content.parts[0].function_call
        tool_name = tool_call.name
        tool_args = tool_call.args
        function_to_ex = tool_map[tool_name]
        tool_output = function_to_ex(**tool_args)

        tool_resp = types.Part.from_function_response(name=tool_name,response={"result":tool_output})

        content.append(types.Content(role='user',parts=[tool_resp]))

        second_response = client.models.generate_content(model='gemini-2.0-flash',contents=content)
        return second_response.text
    else:
        return first_response.text

In [14]:
generate_content("explain quantum computing in 1 line")

'Quantum computing uses quantum mechanics to solve complex problems faster than classical computers.\n'

In [16]:
results = generate_content("what is the weather in Dubai today")
results

LLM decided to make function call 
 id=None args={'city': 'Dubai'} name='get_current_weather'


'The weather in Dubai today is few clouds with a temperature of 300.11 Kelvin.'